# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f853e1f2370>
├── 'a' --> tensor([[-0.2816, -2.4577,  0.0933],
│                   [ 2.6291,  1.5299,  1.5452]])
└── 'x' --> <FastTreeValue 0x7f853e1f6730>
    └── 'c' --> tensor([[-1.7294,  0.6935,  0.1112,  1.7483],
                        [-0.8789,  0.7195, -0.2734, -0.1402],
                        [-0.9207, -0.3307, -0.0130, -1.1925]])

In [4]:
t.a

tensor([[-0.2816, -2.4577,  0.0933],
        [ 2.6291,  1.5299,  1.5452]])

In [5]:
%timeit t.a

65.1 ns ± 1.75 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f853e1f2370>
├── 'a' --> tensor([[-0.0540,  0.1353, -1.3525],
│                   [ 0.5439, -0.5441, -0.5670]])
└── 'x' --> <FastTreeValue 0x7f853e1f6730>
    └── 'c' --> tensor([[-1.7294,  0.6935,  0.1112,  1.7483],
                        [-0.8789,  0.7195, -0.2734, -0.1402],
                        [-0.9207, -0.3307, -0.0130, -1.1925]])

In [7]:
%timeit t.a = new_value

69.2 ns ± 2.47 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.2816, -2.4577,  0.0933],
               [ 2.6291,  1.5299,  1.5452]]),
    x: Batch(
           c: tensor([[-1.7294,  0.6935,  0.1112,  1.7483],
                      [-0.8789,  0.7195, -0.2734, -0.1402],
                      [-0.9207, -0.3307, -0.0130, -1.1925]]),
       ),
)

In [10]:
b.a

tensor([[-0.2816, -2.4577,  0.0933],
        [ 2.6291,  1.5299,  1.5452]])

In [11]:
%timeit b.a

58.3 ns ± 0.209 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.0609, -0.1289,  0.1253],
               [-0.3393, -0.1913, -0.7988]]),
    x: Batch(
           c: tensor([[-1.7294,  0.6935,  0.1112,  1.7483],
                      [-0.8789,  0.7195, -0.2734, -0.1402],
                      [-0.9207, -0.3307, -0.0130, -1.1925]]),
       ),
)

In [13]:
%timeit b.a = new_value

514 ns ± 14.7 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

884 ns ± 8.75 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.7 µs ± 488 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

152 µs ± 2.42 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

151 µs ± 2.53 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f851064d820>
├── 'a' --> tensor([[[-0.2816, -2.4577,  0.0933],
│                    [ 2.6291,  1.5299,  1.5452]],
│           
│                   [[-0.2816, -2.4577,  0.0933],
│                    [ 2.6291,  1.5299,  1.5452]],
│           
│                   [[-0.2816, -2.4577,  0.0933],
│                    [ 2.6291,  1.5299,  1.5452]],
│           
│                   [[-0.2816, -2.4577,  0.0933],
│                    [ 2.6291,  1.5299,  1.5452]],
│           
│                   [[-0.2816, -2.4577,  0.0933],
│                    [ 2.6291,  1.5299,  1.5452]],
│           
│                   [[-0.2816, -2.4577,  0.0933],
│                    [ 2.6291,  1.5299,  1.5452]],
│           
│                   [[-0.2816, -2.4577,  0.0933],
│                    [ 2.6291,  1.5299,  1.5452]],
│           
│                   [[-0.2816, -2.4577,  0.0933],
│                    [ 2.6291,  1.5299,  1.5452]]])
└── 'x' --> <FastTreeValue 0x7f851064df40>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.6 µs ± 359 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f848f4dd190>
├── 'a' --> tensor([[-0.2816, -2.4577,  0.0933],
│                   [ 2.6291,  1.5299,  1.5452],
│                   [-0.2816, -2.4577,  0.0933],
│                   [ 2.6291,  1.5299,  1.5452],
│                   [-0.2816, -2.4577,  0.0933],
│                   [ 2.6291,  1.5299,  1.5452],
│                   [-0.2816, -2.4577,  0.0933],
│                   [ 2.6291,  1.5299,  1.5452],
│                   [-0.2816, -2.4577,  0.0933],
│                   [ 2.6291,  1.5299,  1.5452],
│                   [-0.2816, -2.4577,  0.0933],
│                   [ 2.6291,  1.5299,  1.5452],
│                   [-0.2816, -2.4577,  0.0933],
│                   [ 2.6291,  1.5299,  1.5452],
│                   [-0.2816, -2.4577,  0.0933],
│                   [ 2.6291,  1.5299,  1.5452]])
└── 'x' --> <FastTreeValue 0x7f85108620a0>
    └── 'c' --> tensor([[-1.7294,  0.6935,  0.1112,  1.7483],
                        [-0.8789,  0.7195, -0.2734, -0.1402],
                 

In [23]:
%timeit t_cat(trees)

31.3 µs ± 1.95 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58.6 µs ± 1.83 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.2816, -2.4577,  0.0933],
                [ 2.6291,  1.5299,  1.5452]],
       
               [[-0.2816, -2.4577,  0.0933],
                [ 2.6291,  1.5299,  1.5452]],
       
               [[-0.2816, -2.4577,  0.0933],
                [ 2.6291,  1.5299,  1.5452]],
       
               [[-0.2816, -2.4577,  0.0933],
                [ 2.6291,  1.5299,  1.5452]],
       
               [[-0.2816, -2.4577,  0.0933],
                [ 2.6291,  1.5299,  1.5452]],
       
               [[-0.2816, -2.4577,  0.0933],
                [ 2.6291,  1.5299,  1.5452]],
       
               [[-0.2816, -2.4577,  0.0933],
                [ 2.6291,  1.5299,  1.5452]],
       
               [[-0.2816, -2.4577,  0.0933],
                [ 2.6291,  1.5299,  1.5452]]]),
    x: Batch(
           c: tensor([[[-1.7294,  0.6935,  0.1112,  1.7483],
                       [-0.8789,  0.7195, -0.2734, -0.1402],
                       [-0.9207, -0.3307, -0.0130, -1.1925]],
         

In [26]:
%timeit Batch.stack(batches)

79.4 µs ± 955 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.2816, -2.4577,  0.0933],
               [ 2.6291,  1.5299,  1.5452],
               [-0.2816, -2.4577,  0.0933],
               [ 2.6291,  1.5299,  1.5452],
               [-0.2816, -2.4577,  0.0933],
               [ 2.6291,  1.5299,  1.5452],
               [-0.2816, -2.4577,  0.0933],
               [ 2.6291,  1.5299,  1.5452],
               [-0.2816, -2.4577,  0.0933],
               [ 2.6291,  1.5299,  1.5452],
               [-0.2816, -2.4577,  0.0933],
               [ 2.6291,  1.5299,  1.5452],
               [-0.2816, -2.4577,  0.0933],
               [ 2.6291,  1.5299,  1.5452],
               [-0.2816, -2.4577,  0.0933],
               [ 2.6291,  1.5299,  1.5452]]),
    x: Batch(
           c: tensor([[-1.7294,  0.6935,  0.1112,  1.7483],
                      [-0.8789,  0.7195, -0.2734, -0.1402],
                      [-0.9207, -0.3307, -0.0130, -1.1925],
                      [-1.7294,  0.6935,  0.1112,  1.7483],
                      [-0.8789,  

In [28]:
%timeit Batch.cat(batches)

146 µs ± 3.54 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

340 µs ± 7.76 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
